In [1]:
import requests
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [2]:
url = "https://www.sec.gov/Archives/edgar/data/1800/000104746919000624/a2237733z10-k.htm"

payload = ""
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "Cookie": "bm_mi=2F5AC7DBEC90CA72B7048BFB724142E4~YAAQVCR+aDfBglWNAQAAXdbhXxZRJ4RnkkNmfKWUeKc1QmGH7v3TlPngbOdNdESvidaNFbsxYRDuqNOeHf7o1570sBMa6264ltivVUhPKdRHSGYpVZ2XMWAD/nsmFsSdGcaKYGWMYw5uwfMf/PBPBlZjEQFik5jnd6rWMARaXnpOcYh9njmeVVV519UStSmkmZI5vOvjdoluWej0Z2CxX3Tvw6lNgc1tGVpPsmufVg31H7mrJXYEd2r8dgK/8ByPdjyyHV9vmHGVXWgtIYQEPdFuSZTZcKKd+BnNxBwJMemL5bYVUglFp1bWgOrjeN1N3ix8AELEMXzUgBOh3O3bOm20cMppfRlldfsle3K8Ih4OPhQV7X2gCBWcP/IF9xpVVVUCvbQc36+ZPkQeYwCpp1ZbDy4=~1; bm_sv=6FE42A06849FED4835CF97262309E48E~YAAQVCR+aELBglWNAQAA9N3hXxbgrPGTXP4dLr20mIrEPGqqt/fAg73yGCttn0ty9QTvryHyxJIj5/Xk9bdr1pg+N8DKmsja3McwAOPL66B3PbRpOXDni0I/7hj2onuFL1tySgSQdD5H2wbu2hHzLereVi/obIf/64d3G6DMRmi21FiZaZXw0bKx8Dcbey35f4/a0B3//5sv5dyHH022pza3IR0waWQFv9Up4U5Z1kAbE6dcCBfgw81JPrQ0CA==~1; ak_bmsc=A56127CD072C24354C54D9D888F73150~000000000000000000000000000000~YAAQVCR+aB3CglWNAQAAPE7iXxZxAH8EfBgLMoBK20p82jHscZyuohv2ZDf4igBKSFrl/lk2WxTlHBCZjxlNf0g7+4qhiHtb+iiwCO6dsttHAG4aLtjhcRUqGypjDKB5LdHa2gdTTYL6d+zDand7hgzuYEMTmjSlIXjsdiIpL5cRkkkc62gsj4VYGbKiS7sXf8ZJrpwcTVflEsete1zfSn9uR5h8ydDbzbLPj/YOTNBCNU2lE61l66SPhy3c3LkvFWNB33V1iZVX/r7AqomFCHZQrZiEejEOczLuLEfLyEE/ol0OlL5UgHM/KIfQtfG7xhej2JoGiKzIBRyzQT7iChekHOjlzZgF0IvCURxPvw4bLD14OlSOsh9QYMBkH6MEs8csRoOCbuBdYFKkXESQ+dC0QkyTlDgDbzv61UZ6u/Tzsp3AlkDvWHW2FLNVdZE8G47QP0auueW3sMi84Y3n4reVALhIZUHywXQ6w4F4K3P3fQwcU+F/qOFevEsQU60mNxbPeEc=",
    "If-Modified-Since": "Fri, 22 Feb 2019 21 17:19 GMT",
    "Sec-Ch-Ua": '"Not A(Brand";v="99", "Brave";v="121", "Chromium";v="121"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"macOS"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Sec-Gpc": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

response = requests.request("GET", url, data=payload, headers=headers)

In [3]:
# soup = BeautifulSoup(response.text, "html.parser")
soup = BeautifulSoup(response.text, "lxml")

In [44]:
# define a dictionary that will house all filings.
master_filings_dict = {}

# let's use the accession number as the key. This 
# accession_number = '0001104659-04-027382'

# add a new level to our master_filing_dict, this will also be a dictionary.
master_filings_dict[accession_number] = {}

# this dictionary will contain two keys, the sec header content, and a documents key.
master_filings_dict[accession_number]['sec_header_content'] = {}
master_filings_dict[accession_number]['filing_documents'] = None

# grab the sec-header tag, so we can store it in the master filing dictionary.
sec_header_tag = soup.find('sec-header')

# store the tag in the dictionary just as is.
master_filings_dict[accession_number]['sec_header_content']['sec_header_code'] = sec_header_tag

# display the sec header tag, so you can see how it looks.
display(sec_header_tag)

None

In [5]:
sec_header_tag = soup.find('sec-header')

In [50]:
soup

<document>
<type>10-K
<sequence>1
<filename>a2237733z10-k.htm
<description>10-K
<text>
<html>
<head>
</head>
<body bgcolor="#FFFFFF" link="BLUE" vlink="PURPLE">
<br/>
<p style="font-family:times;text-align:justify"><font size="2">
<!-- COMMAND=ADD_BASECOLOR,"#000000" -->
</font> <font size="2"><b>
<!-- COMMAND=ADD_DEFAULTFONT,"font-family:times;" -->
<!-- COMMAND=ADD_TABLESHADECOLOR,"#CCEEFF" -->
<!-- COMMAND=ADD_STABLERULES,"border-bottom:solid #000000 1.0pt;" -->
<!-- COMMAND=ADD_DTABLERULES,"border-bottom:double #000000 2.25pt;" -->
<!-- COMMAND=ADD_SCRTABLERULES,"border-bottom:solid #000000 1.0pt;margin-bottom:0pt;" -->
<!-- COMMAND=ADD_DCRTABLERULES,"border-bottom:double #000000 2.25pt;margin-bottom:0pt;" -->
<!-- PARA=JUSTIFY -->
</b></font></p>
<p style="font-family:times;text-align:justify"><font size="2"><b>  <br/> </b></font></p>
<p style="font-family:times;text-align:justify"><font size="2"><b> <div style="width:100%;border-top:solid #000000 3.0pt;padding:0in 0in 0in 0in;fon

In [47]:
soup.find('sec-header')

In [41]:
# initalize the dictionary that will house all of our documents
master_document_dict = {}

# find all the documents in the filing.
for filing_document in soup.find_all('document'):
    
    # define the document type, found under the <type> tag, this will serve as our key for the dictionary.
    document_id = filing_document.type.find(text=True, recursive=False).strip()
    
    # here are the other parts if you want them.
    document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
    document_filename = filing_document.filename.find(text=True, recursive=False).strip()
    document_description = filing_document.description.find(text=True, recursive=False).strip()
    
    # initalize our document dictionary
    master_document_dict[document_id] = {}
    
    # add the different parts, we parsed up above.
    master_document_dict[document_id]['document_sequence'] = document_sequence
    master_document_dict[document_id]['document_filename'] = document_filename
    master_document_dict[document_id]['document_description'] = document_description
    
    # store the document itself, this portion extracts the HTML code. We will have to reparse it later.
    master_document_dict[document_id]['document_code'] = filing_document.extract()
    
    
    # grab the text portion of the document, this will be used to split the document into pages.
    filing_doc_text = filing_document.find('text').extract()

    
    # find all the thematic breaks, these help define page numbers and page breaks.
    all_thematic_breaks = filing_doc_text.find_all('hr',{'width':'100%'})
    
    # grab all the page numbers, first one is usually blank
    all_page_numbers = [thematic_break.parent.parent.previous_sibling.previous_sibling.get_text(strip=True) 
                        for thematic_break in all_thematic_breaks]
    
     # determine the number of pages, will be used for the upcoming if conditions.
    length_of_page_numbers = len(all_page_numbers)
    
    # as long as there are numbers to change then proceed.
    if length_of_page_numbers > 0:
        
        # grab the last number
        previous_number = all_page_numbers[-1]
        
        # initalize a new list
        all_page_numbers_cleaned = []
        
        # loop through the old list in reverse order.
        for number in reversed(all_page_numbers):
            
            # if it's blank proceed to cleaning.
            if number == '':
                
                # the tricky part, there are three scenarios.

                # the previous one we looped was 0 or 1.
                if previous_number == '1' or previous_number == '0':
                    
                    # in this case, it means this is a "new section", so restart at 0.
                    all_page_numbers_cleaned.append(str(0))
                    
                    # reset the page number and the previous number.
                    length_of_page_numbers = length_of_page_numbers - 1
                    previous_number = '0'
                
                # the previous one we looped it wasn't either of those.
                else:
                    
                    # if it was blank, take the current length, subtract 1, and add it to the list.
                    all_page_numbers_cleaned.append(str(length_of_page_numbers - 1))
                    
                    # reset the page number and the previous number.
                    length_of_page_numbers = length_of_page_numbers - 1
                    previous_number = number

            else:
                
                # add the number to the list.
                all_page_numbers_cleaned.append(number)
                
                # reset the page number and the previous number.
                length_of_page_numbers = length_of_page_numbers - 1
                previous_number = number
    else:
        
        # make sure that it has a page number even if there are none, just have it equal 0
        all_page_numbers_cleaned = ['0']
    
    # have the page numbers be the cleaned ones, in reversed order.
    all_page_numbers = list(reversed(all_page_numbers_cleaned))
    
    # store the page_numbers
    master_document_dict[document_id]['page_numbers'] = all_page_numbers
    
    
    '''
        -------------------------------
          THE OPTIONAL CODE HAS ENDED
        -------------------------------
    
        This next portion of code is really what made this all possible. Up above you saw I grabbed all the thematic
        breaks from our document because they sever as natural page breaks. Without those thematic breaks I'm not sure
        if this would be such an easy process. It's not to say we couldn't break it into pages, but I would bet the code
        would be more complex.
    
    '''
    
    
    # convert all thematic breaks to a string so it can be used for parsing
    all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]
    
    # prep the document text for splitting, this means converting it to a string.
    filing_doc_string = str(filing_doc_text)

    
    # handle the case where there are thematic breaks.
    if len(all_thematic_breaks) > 0:
    
        # define the regex delimiter pattern, this would just be all of our thematic breaks.
        regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))

        # split the document along each thematic break.
        split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)

        # store the document itself
        master_document_dict[document_id]['pages_code'] = split_filing_string

    # handle the case where there are no thematic breaks.
    elif len(all_thematic_breaks) == 0:

        # handles so it will display correctly.
        split_filing_string = all_thematic_breaks
        
        # store the document as is, since there are no thematic breaks. In other words, no splitting.
        master_document_dict[document_id]['pages_code'] = [filing_doc_string]
    

    # display some information to the user.
    print('-'*80)
    print('The document {} was parsed.'.format(document_id))
    print('There was {} page(s) found.'.format(len(all_page_numbers)))
    print('There was {} thematic breaks(s) found.'.format(len(all_thematic_breaks)))
    

# store the documents in the master_filing_dictionary.
master_filings_dict[accession_number]['filing_documents'] = master_document_dict

print('-'*80)
print('All the documents for filing {} were parsed and stored.'.format(accession_number))

NameError: name 'master_filings_dict' is not defined

In [40]:
all_page_numbers

NameError: name 'all_page_numbers' is not defined

In [23]:
soup.find_all("b")

[<b>
 <!-- COMMAND=ADD_DEFAULTFONT,"font-family:times;" -->
 <!-- COMMAND=ADD_TABLESHADECOLOR,"#CCEEFF" -->
 <!-- COMMAND=ADD_STABLERULES,"border-bottom:solid #000000 1.0pt;" -->
 <!-- COMMAND=ADD_DTABLERULES,"border-bottom:double #000000 2.25pt;" -->
 <!-- COMMAND=ADD_SCRTABLERULES,"border-bottom:solid #000000 1.0pt;margin-bottom:0pt;" -->
 <!-- COMMAND=ADD_DCRTABLERULES,"border-bottom:double #000000 2.25pt;margin-bottom:0pt;" -->
 <!-- PARA=JUSTIFY -->
 </b>,
 <b>  <br/> </b>,
 <b> <div style="width:100%;border-top:solid #000000 3.0pt;padding:0in 0in 0in 0in;font-size:3.0pt;"></div>
 <div style="width:100%;border-top:solid #000000 1.0pt;padding:0in 0in 0in 0in;font-size:4.0pt;"></div> </b>,
 <b>UNITED STATES<br/>
 SECURITIES AND EXCHANGE COMMISSION<br/> </b>,
 <b>WASHINGTON, D. C. 20549</b>,
 <b>FORM 10-K  </b>,
 <b>(MARK ONE)</b>,
 <b><font face="WINGDINGS">ý</font></b>,
 <b>ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF<br/>
 THE SECURITIES EXCHANGE ACT OF 1934</b>,
 <b> OR</b>,


# JSON Submissions

In [52]:
url = "https://data.sec.gov/submissions/CIK0000001800.json"
r = requests.request("GET", url, data=payload, headers=headers)

In [79]:
pd.DataFrame(r.json()["filings"]["recent"]).query("form=='10-K'").reset_index(drop=True)

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0001628280-23-004026,2023-02-17,2022-12-31,2023-02-17T16:09:44.000Z,34,10-K,001-02189,23642562,,19304379,1,1,abt-20221231.htm,10-K
1,0001104659-22-025141,2022-02-18,2021-12-31,2022-02-18T16:02:57.000Z,34,10-K,001-02189,22652622,,18785134,1,1,abt-20211231x10k.htm,10-K
2,0001104659-21-025751,2021-02-19,2020-12-31,2021-02-19T16:10:04.000Z,34,10-K,001-02189,21655651,,21381769,1,1,abt-20201231x10k.htm,10-K
3,0001104659-20-023904,2020-02-21,2019-12-31,2020-02-21T16:28:23.000Z,34,10-K,001-02189,20640433,,20812248,1,1,abt-20191231x10k59d41b.htm,10-K
4,0001047469-19-000624,2019-02-22,2018-12-31,2019-02-22T16:13:21.000Z,34,10-K,001-02189,19625847,,19514449,1,0,a2237733z10-k.htm,10-K
5,0001047469-18-000856,2018-02-16,2017-12-31,2018-02-16T16:04:37.000Z,34,10-K,001-02189,18621190,,19787072,1,0,a2234264z10-k.htm,10-K
6,0001047469-17-000744,2017-02-17,2016-12-31,2017-02-17T16:17:24.000Z,34,10-K,001-02189,17621570,,20005840,1,0,a2230875z10-k.htm,10-K


In [64]:
pd.json_normalize(r.json()["filings"]).T

,0
files,[{'name': 'CIK0000001800-submissions-001.json'...
recent.accessionNumber,"[0001104659-24-007725, 0001973141-24-000050, 0..."
recent.filingDate,"[2024-01-29, 2024-01-29, 2024-01-26, 2024-01-2..."
recent.reportDate,"[2024-01-29, , , 2024-01-24, 2023-12-31, 2023-..."
recent.acceptanceDateTime,"[2024-01-29T16:10:51.000Z, 2024-01-29T16:04:19..."
recent.act,"[34, 33, 34, 34, , , , , , 33, 34, , 33, 34, ,..."
recent.form,"[8-K, 144, SC 13G/A, 8-K, 4, 4, 4, 4, 4, 144, ..."
recent.fileNumber,"[001-02189, 001-02189, 005-10519, 001-02189, ,..."
recent.filmNumber,"[24573755, 24573615, 24565741, 24554251, , , ,..."
recent.items,"[1.01,1.02,2.03, , , 2.02,9.01, , , , , , , , ..."


In [82]:
# base URL for the SEC EDGAR browser
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

# define our parameters dictionary
param_dict = {'action':'getcompany',
              'CIK':'1265107',
              'type':'10-k',
              'dateb':'20190101',
              'owner':'exclude',
              'start':'',
              'output':'',
              'count':'100'}

# request the url, and then parse the response.
response = requests.get(url = endpoint, params = param_dict)
soup = BeautifulSoup(response.content, 'html.parser')

# Let the user know it was successful.
print('Request Successful')
print(response.url)

Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1265107&type=10-k&dateb=20190101&owner=exclude&start=&output=&count=100
